In [5]:
import pandas as pd

spam_data = pd.read_csv('spam.csv',encoding = 'latin-1')

display(spam_data.head())


,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [6]:
print(spam_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   v1          5572 non-null   object
 1   v2          5572 non-null   object
 2   Unnamed: 2  50 non-null     object
 3   Unnamed: 3  12 non-null     object
 4   Unnamed: 4  6 non-null      object
dtypes: object(5)
memory usage: 217.8+ KB
None


In [7]:
spam_data_cleaned = spam_data.drop(columns=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'])

print(spam_data_cleaned.head())

     v1                                                 v2
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re
import nltk

nltk.download('stopwords')

def clean_text(text):
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = text.lower()
    stop_words = set(stopwords.words('english'))
    text = ' '.join([word for word in text.split() if word not in stop_words])
    stemmer = PorterStemmer()
    text = ' '.join([stemmer.stem(word) for word in text.split()])
    return text

cleaned_texts = spam_data_cleaned['v2'].apply(clean_text)

print(cleaned_texts.head())

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Caxy\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


0    go jurong point crazi avail bugi n great world...
1                                ok lar joke wif u oni
2    free entri wkli comp win fa cup final tkt st m...
3                  u dun say earli hor u c alreadi say
4                 nah think goe usf live around though
Name: v2, dtype: object


In [9]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(cleaned_texts)
y = spam_data_cleaned['v1'].map({'ham': 0, 'spam': 1})  

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print('Training set shape:', X_train.shape)
print('Testing set shape:', X_test.shape)

Training set shape: (4457, 6221)
Testing set shape: (1115, 6221)


In [12]:
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', random_state=42)
svm_model.fit(X_train, y_train)

y_pred_svm = svm_model.predict(X_test)

accuracy_svm = accuracy_score(y_test, y_pred_svm)

report_svm = classification_report(y_test, y_pred_svm, target_names=['Ham', 'Spam'])

print('SVM Accuracy:', accuracy_svm)
print('SVM Classification Report:\n', report_svm)

SVM Accuracy: 0.9775784753363229
SVM Classification Report:
               precision    recall  f1-score   support

         Ham       0.98      1.00      0.99       965
        Spam       0.98      0.85      0.91       150

    accuracy                           0.98      1115
   macro avg       0.98      0.93      0.95      1115
weighted avg       0.98      0.98      0.98      1115



In [13]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
import numpy as np

tokenizer = Tokenizer()
tokenizer.fit_on_texts(cleaned_texts)
sequences = tokenizer.texts_to_sequences(cleaned_texts)

X = pad_sequences(sequences)

le = LabelEncoder()
Y = le.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=100, input_length=X.shape[1]))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=5, batch_size=64, validation_split=0.1)

loss, accuracy = model.evaluate(X_test, y_test)
print('RNN Model Accuracy:', accuracy)




Epoch 1/5


63/63 [==============================] - 16s 165ms/step - loss: 0.2317 - accuracy: 0.9247 - val_loss: 0.1197 - val_accuracy: 0.9596
Epoch 2/5
63/63 [==============================] - 10s 152ms/step - loss: 0.0375 - accuracy: 0.9905 - val_loss: 0.0899 - val_accuracy: 0.9753
Epoch 3/5
63/63 [==============================] - 10s 162ms/step - loss: 0.0135 - accuracy: 0.9965 - val_loss: 0.1011 - val_accuracy: 0.9664
Epoch 4/5
63/63 [==============================] - 11s 173ms/step - loss: 0.0079 - accuracy: 0.9975 - val_loss: 0.1061 - val_accuracy: 0.9686
Epoch 5/5
35/35 [==============================] - 1s 14ms/step - loss: 0.0600 - accuracy: 0.9812
RNN Model Accuracy: 0.9811659455299377
